In [1]:
import torch
from utils import *       #changed
from models import TransformerLM, TransformerConditionedLM
import pickle
from imageio import imread
from PIL import Image
import torch.nn.functional as F

In [25]:
import yaml
# /net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/bleu-4_BEST_checkpoint_coco_2_cap_per_img_1_min_word_freq_gpu.pth.tar
# checkpoint_path = "../../saved_model/I2U/trimmed_mapping_SpeakerALL/Trial_3/"
checkpoint_path = "../../saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/"

with open(checkpoint_path + 'config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

checkpoint = checkpoint_path + "bleu-4_BEST_checkpoint_coco_2_cap_per_img_1_min_word_freq_gpu.pth.tar"
dir_name = config["i2u"]["dir_name"]
model_params = config["i2u"]["model_params"]
train_params = config["i2u"]["train_params"]
data_folder = f'../../data/processed/{dir_name}/'  # folder with data files saved by create_input_files.py
# data_name = 'coco_4_cap_per_img_5_min_word_freq'  # base name shared by data files
#data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files
data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
# device = torch.device("cpu")

In [27]:
word_map_file = os.path.join(data_folder, 'WORDMAP_' + data_name + '.json')
with open(word_map_file, 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}
# Initialize / load checkpoint
model_params['vocab_size'] = len(word_map)
model = TransformerConditionedLM(**model_params)
optimizer = getattr(torch.optim, train_params["optimizer"])(model.parameters(), lr=train_params["lr"])

model, optimizer, start_epoch, best_bleu4, best_accuracy = load_checkpoint(checkpoint, model, optimizer, device)
model.eval()
model.to(device)

Loading checkpoint from ../../saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/bleu-4_BEST_checkpoint_coco_2_cap_per_img_1_min_word_freq_gpu.pth.tar


TransformerConditionedLM(
  (embed): Embedding(953, 1024)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (LM_decoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=4096, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_fea

In [28]:
start_unit = word_map["<start>"]
end_unit = word_map["<end>"]
vocab_size = len(word_map)
max_len = 150

In [32]:
# test_img = "/net/papilio/storage2/yhaoyuan/transformer_I2U/data/processed/encodec1/test_image_paths.pickle"
# test_img_caps = "/net/papilio/storage2/yhaoyuan/transformer_I2U/data/processed/encodec1/test_image_captions.pickle"
# test_img = "../../data/processed/trimmed_mapping_SpeakerALL/test_image_paths.pickle"
# test_img_caps = "../../data/processed/trimmed_mapping_SpeakerALL/test_image_captions.pickle"
test_img = "../../data/processed/trimmed_mapping_SpeakerALL_Color/test_image_paths.pickle"
test_img_caps = "../../data/processed/trimmed_mapping_SpeakerALL_Color/test_image_captions.pickle"
with open(test_img, "rb") as f:
    test_img = pickle.load(f)
with open(test_img_caps, "rb") as f:
    test_img_caps = pickle.load(f)

In [33]:
ref = [word_map[str(unit)] for unit in test_img_caps[0][0]]

In [34]:
import torchvision.transforms as transforms
def read_img(img_path):
    img = imread(img_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    # img = imresize(img, (256, 256))
    resolution = int(config['data']['image_resolution'])
    img = np.array(Image.fromarray(img).resize((resolution, resolution)))
    img = img.transpose(2, 0, 1)
    assert img.shape == (3, resolution, resolution)
    assert np.max(img) <= 255
    img = img / 255.
    img = torch.FloatTensor(img)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)
    return img

In [35]:
# read_img("/net/tateha/storage2/database/spolacq/FoodImagesA/11_avocado/avocado_bl3_07.jpg")

In [36]:
def get_seq_words(img_path, max_len, beam_size):
    # print(img_path)
    img = read_img(img_path=img_path)
    # img = torch.Tensor(img)
    img = img.unsqueeze(dim = 0)
    seqs = model.decode(img, start_unit, end_unit, max_len, beam_size)
    return seqs

In [37]:
seqs_dict = {}
words_dict = {}
for img in tqdm(test_img):
    seqs = get_seq_words(img_path = img, max_len=150, beam_size=10)
    seqs_dict[img] = seqs
    words_dict[img] = [rev_word_map[ind] for ind in seqs]
with open("seq_dict.json", "w") as f:
    json.dump(seqs_dict, f)
with open("words_dict.json", "w") as f:
    json.dump(words_dict, f)

  0%|          | 0/415 [00:00<?, ?it/s]/tmp/ipykernel_1201566/2626450078.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(img_path)
100%|██████████| 415/415 [08:05<00:00,  1.17s/it]


In [19]:
words = [rev_word_map[ind] for ind in seqs]
#sequence = np.array(text_to_sequence(' '.join(words[1:-1]), ['english_cleaners']))[None, :]

In [22]:
# with open("test_word.json", "w") as f:
#     json.dump(words, f)

In [16]:
img = read_img(img_path=test_img[0])
img = torch.Tensor(img)
img = img.unsqueeze(dim = 0)
img = img.to(device)
print(img.dim())
img_features, gx = model.image_encoder(img)
gx, img_features = model.refine_encoder(img_features)

4


/tmp/ipykernel_484196/2858467658.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(img_path)


In [ ]:
k = 5
k_prev_words = torch.LongTensor([[start_unit]] * k).to(device)
seqs = k_prev_words  # (k, 1)
# Tensor to store top k sequences' scores; now they're just 0
top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)
# Lists to store completed sequences and scores
complete_seqs = list()
complete_seqs_scores = list()
# Start decoding
step = 1
gx_ex = gx.expand(k, 1, 1024)

In [19]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
while True:
    x = model.embed(seqs)
    x = model.pos_encoder(x)
    decoder_input = torch.cat([gx_ex, x], dim = 1)
    img_features_ex = img_features.expand(k, 196, 1024)
    output = model.decoder(decoder_input, img_features_ex)
    scores = model.classifier(output[:,-1,:])
    scores = F.log_softmax(scores, dim=1)
    # Add
    scores = top_k_scores.expand_as(scores) + scores  # (s, vocab_size)
    if step == 1:
        top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
    else:
        # Unroll and find top scores, and their unrolled indices
        top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)

    prev_word_inds = torch.div(top_k_words, vocab_size, rounding_mode="floor")
    next_word_inds = top_k_words % vocab_size  # (s)
    seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)
    incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if next_word != end_unit]
    complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))
    if len(complete_inds) > 0:
        complete_seqs.extend(seqs[complete_inds].tolist())
        complete_seqs_scores.extend(top_k_scores[complete_inds])
    k -= len(complete_inds)  # reduce beam length accordingly
    # Proceed with incomplete sequences
    if k == 0:
        break
    seqs = seqs[incomplete_inds]
    img_features = img_features[prev_word_inds[incomplete_inds]]
    top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
    k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)
    # Break if things have been going on too long
    print(seqs.shape)
    if step > max_len:
        break
    step += 1

In [ ]:
if len(complete_seqs_scores) != 0:
    i = complete_seqs_scores.index(max(complete_seqs_scores))
    seq = complete_seqs[i]
else:
    seq = []

In [8]:
a = torch.FloatTensor(5,10)

In [16]:
a[:, -1] = 1

In [17]:
a

tensor([[ 2.2245e-33,  4.5881e-41,  6.7815e-35,  0.0000e+00,  1.4013e-45,
          0.0000e+00,  4.4701e-43,  5.3595e-38,  3.0696e-34,  1.0000e+00],
        [ 3.0696e-34,  0.0000e+00,  4.3300e-43,  0.0000e+00,  4.3440e-43,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  3.0666e-34,
          0.0000e+00,  1.4013e-45,  5.6052e-45,  3.0699e-34,  1.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  2.2421e-43,  0.0000e+00,  2.2421e-43,  1.0000e+00],
        [-1.0318e+01,  4.5880e-41,  0.0000e+00,  0.0000e+00,  1.4013e-45,
          0.0000e+00,  3.7835e-43,  0.0000e+00,  3.0696e-34,  1.0000e+00]])